In [ ]:
# pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import requests
from bs4 import BeautifulSoup
import time

# print artist, genre, and download link

In [5]:
# The URL of the website you want to scrape
url = 'https://cambridge-mt.com/ms/mtk/#topAnchor'

# Send an HTTP request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Assuming 'c-mtk__artist' is the class name for each artist's container
    artists = soup.find_all('div', class_='c-mtk__artist')

    total_tracks = []

    for artist in artists:
        # Extract the artist name and genre. Adjust based on actual page content.
        name = artist.find('h4').text.strip()
        genre = artist.find('span', class_='m-container__title-bar-item').text.strip()

        # Print the artist's name and genre
        print(f'Artist Name: {name}')
        print(f'Genre: {genre}')

        # Extract and print track names and download links
        tracks = artist.find_all('li', class_='m-mtk-track')
        total_tracks.extend(tracks)

        for track in tracks:
            track_name = track.find('span', class_='m-mtk-track__name').text.strip()
            print(f'Track Name: {track_name}')

            downloads = track.find_all('a')
            for download in downloads:
                if download.get('href').startswith('http'):
                    print(f"Download Link: {download['href']}")
    print(len(total_tracks))
else:
    print(f'Failed to retrieve the webpage. Status code: {response.status_code}')


Artist Name: 3D-MARCo Project
Genre: Multimiking Comparison Recording
Track Name: 'A Capella'
Download Link: https://discussion.cambridge-mt.com/forumdisplay.php?fid=867
Download Link: https://zenodo.org/record/3477602/files/09%203D-MARCo%20Samples_Acappella.zip?download=1
Download Link: https://previews.cambridge-mt.com/3DMARCoACapella_Full_Preview.mp3
Track Name: 'Piano Solo 1'
Download Link: https://discussion.cambridge-mt.com/forumdisplay.php?fid=825
Download Link: https://zenodo.org/record/3477602/files/07%203D-MARCo%20Samples_Piano%20solo%201.zip?download=1
Download Link: https://previews.cambridge-mt.com/3DMARCoPianoSolo1_Full_Preview.mp3
Track Name: 'String Quartet'
Download Link: https://discussion.cambridge-mt.com/forumdisplay.php?fid=808
Download Link: https://zenodo.org/record/3477602/files/04%203D-MARCo%20Samples_Quartet.zip?download=1
Download Link: https://previews.cambridge-mt.com/3DMARCoStringQuartet_Full_Preview.mp3
Artist Name: The Abletones Big Band
Genre: Big Band 

# Download files, folder name format: Track - Genre - Artist

In [ ]:
def sanitize_filename(name):
    """
    Removes any character from the name that is not alphanumeric, a space, an underscore, or a hyphen.
    """
    return "".join(c for c in name if c.isalnum() or c in [' ', '_', '-']).rstrip()



def download_file(url, directory, subfolder_name):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    } # This is to mimic a browser, necessary step to avoid being refused by the website

    # Create the full directory path if it doesn't exist
    full_path = os.path.join(directory, subfolder_name)
    if not os.path.exists(full_path):
        os.makedirs(full_path)

    # Get the last part of the URL to use as the file name
    local_filename = url.split('/')[-1]
    local_filepath = os.path.join(full_path, local_filename)

    # Stream download the file to handle large files
    with requests.get(url, headers=headers, stream=True) as r:
        r.raise_for_status()
        with open(local_filepath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filepath


# Base directory where files will be saved
base_directory = './project_data/'

# URL of the website to scrape
url = 'https://cambridge-mt.com/ms/mtk/#topAnchor'

# Send HTTP request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    artists = soup.find_all('div', class_='c-mtk__artist')

    for artist in artists:
        artist_name = artist.find('h4').text.strip()
        genre = artist.find('span', class_='m-container__title-bar-item').text.strip()

        tracks = artist.find_all('li', class_='m-mtk-track')
        for track in tracks:
            track_name = track.find('span', class_='m-mtk-track__name').text.strip()
            # Sanitize each part to be file system safe
            safe_artist_name = sanitize_filename(artist_name)
            safe_genre = sanitize_filename(genre)
            safe_track_name = sanitize_filename(track_name)
            # Combine artist name, genre, and track name for the folder
            folder_name = f"{safe_track_name} - {safe_genre} - {safe_artist_name}"

            downloads = track.find_all('a')
            if downloads:
                last_download = downloads[-1]
                if last_download.get('href').startswith('http'):
                    file_url = last_download['href']
                    print(f'Downloading {file_url}...')

                    # Introduce a delay before making the download request, better to keep it
                    time.sleep(0.3)

                    download_file(file_url, base_directory, folder_name)
                    print(f'Downloaded to: {os.path.join(base_directory, folder_name)}')

# The above code is to download only mixed mp3, if need all files, use the following code:

#             for download in downloads:
#                 if download.get('href').startswith('http'):
#                     print(f'Downloading {file_url}')
#                     time.sleep(0.3)
#                     file_url = download['href']
#                     # Download the file into the specified folder
#                     download_file(file_url, base_directory, folder_name)
#                     print(f'Downloaded {file_url} \n    to: {folder_name}')

else:
    print(f'Failed to retrieve the webpage. Status code: {response.status_code}')


Downloaded to: ./project_data/A Capella - Multimiking Comparison Recording - 3D-MARCo Project
Downloaded to: ./project_data/Piano Solo 1 - Multimiking Comparison Recording - 3D-MARCo Project
Downloaded to: ./project_data/String Quartet - Multimiking Comparison Recording - 3D-MARCo Project
Downloaded to: ./project_data/Corine Corine - Big Band Jazz - The Abletones Big Band
Downloaded to: ./project_data/Song Of India - Big Band Jazz - The Abletones Big Band
Downloaded to: ./project_data/Rumba Chonta - Afrocolombian Folk Instrumental - Alejo Granados
Downloaded to: ./project_data/Dead Roses - Acoustic Folk-Pop - Andrew Cole


KeyboardInterrupt: 

# Add tagging, still working on tagging tools

In [ ]:
!pip install librosa

  Obtaining dependency information for librosa from https://files.pythonhosted.org/packages/e2/a2/4f639c1168d7aada749a896afb4892a831e2041bebdcf636aebfe9e86556/librosa-0.10.1-py3-none-any.whl.metadata
  Obtaining dependency information for audioread>=2.1.9 from https://files.pythonhosted.org/packages/57/8d/30aa32745af16af0a9a650115fbe81bde7c610ed5c21b381fca0196f3a7f/audioread-3.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for pooch>=1.0 from https://files.pythonhosted.org/packages/f4/72/8ae0f1ba4ce6a4f6d4d01a60a9fdf690fde188c45c1872b0b4ddb0607ace/pooch-1.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for soxr>=0.3.2 from https://files.pythonhosted.org/packages/d0/79/c93daceac24cd6830333d9f3d04716115240ef3be4e8d8ece511afbcf417/soxr-0.3.7-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 4.9 MB/s eta 0:00:00
   ━━━

In [ ]:
import os
import time
import requests
from bs4 import BeautifulSoup
import librosa
import numpy as np

def sanitize_filename(name):
    """Removes characters that are not alphanumeric, spaces, underscores, or hyphens."""
    return "".join(c for c in name if c.isalnum() or c in [' ', '_', '-']).rstrip()

def download_file(url, directory, subfolder_name):
    """Downloads a file from a URL into a specified directory and subfolder."""
    headers = {
        'User-Agent': 'Mozilla/5.0'
    }
    full_path = os.path.join(directory, subfolder_name)
    if not os.path.exists(full_path):
        os.makedirs(full_path)
    local_filename = url.split('/')[-1]
    local_filepath = os.path.join(full_path, local_filename)
    with requests.get(url, headers=headers, stream=True) as r:
        r.raise_for_status()
        with open(local_filepath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filepath

def analyze_audio(file_path):
    """Loads an audio file and extracts various features, if the file is an audio file."""
    if os.path.isfile(file_path) and file_path.endswith('.mp3'):
        y, sr = librosa.load(file_path)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        mfccs = librosa.feature.mfcc(y=y, sr=sr)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        print(f"Analyzing {os.path.basename(file_path)}")
        print('Tempo: {:.2f}'.format(tempo))
        print('MFCCs:', np.mean(mfccs, axis=1))
        print('Chroma:', np.mean(chroma, axis=1))
        print('Spectral Centroid:', np.mean(centroid))
    else:
        print(f"Skipping non-audio or non-existent file: {file_path}")


# Define base directory for file saving
base_directory = './project_data/'

# Specify URL for web scraping
url = 'https://cambridge-mt.com/ms/mtk/#topAnchor'

response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    artists = soup.find_all('div', class_='c-mtk__artist')

    for artist in artists:
        artist_name = artist.find('h4').text.strip()
        genre = artist.find('span', class_='m-container__title-bar-item').text.strip()

        tracks = artist.find_all('li', class_='m-mtk-track')
        for track in tracks:
            track_name = track.find('span', class_='m-mtk-track__name').text.strip()
            safe_artist_name = sanitize_filename(artist_name)
            safe_genre = sanitize_filename(genre)
            safe_track_name = sanitize_filename(track_name)
            folder_name = f"{safe_track_name} - {safe_genre} - {safe_artist_name}"

            downloads = track.find_all('a')
            if downloads:
                last_download = downloads[-1]
                if last_download.get('href').startswith('http'):
                    file_url = last_download['href']
                    print(f'\n\nDownloading {file_url}...')
                    time.sleep(0.3)  # Delay to prevent server overload
                    downloaded_file_path = download_file(file_url, base_directory, folder_name)
                    print(f'Downloaded to: {downloaded_file_path}')
                    analyze_audio(downloaded_file_path)
else:
    print(f'Failed to retrieve the webpage. Status code: {response.status_code}')




Downloaded to: ./project_data/A Capella - Multimiking Comparison Recording - 3D-MARCo Project/3DMARCoACapella_Full_Preview.mp3
Analyzing 3DMARCoACapella_Full_Preview.mp3
Tempo: 123.05
MFCCs: [-2.2991647e+02  1.3474767e+02 -5.7210402e+00  7.7837362e+00
 -1.0548357e+01 -7.3074708e+00  2.8823266e+00 -1.2412205e+01
 -1.0997510e+01  3.2086773e+00 -7.5156274e+00  8.9691365e-01
 -7.2078886e+00 -4.5059210e-01 -9.8696493e-02  1.1369779e+00
  5.1521153e+00  1.5355471e+00  2.0241699e+00  2.8527117e+00]
Chroma: [0.23834088 0.2254315  0.40115714 0.20243885 0.17856352 0.32671425
 0.24808107 0.49164188 0.20976748 0.3224477  0.4944073  0.20478803]
Spectral Centroid: 1517.487831600645


Downloaded to: ./project_data/Piano Solo 1 - Multimiking Comparison Recording - 3D-MARCo Project/3DMARCoPianoSolo1_Full_Preview.mp3
Analyzing 3DMARCoPianoSolo1_Full_Preview.mp3
Tempo: 112.35
MFCCs: [-340.7593     189.45322     14.989884     2.9804606    6.298715
   -0.7649379   -7.846277   -10.324892    -8.06236     -

KeyboardInterrupt: 